# A full business solution



Create a product that builds a Brochure for an Airbnb company and put it out in different languages

In [1]:
# import necessary libraries

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI


In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
airbnb = Website("https://www.airbnb.com/")
airbnb.links

['#site-content',
 '/',
 '/homes',
 '/experiences',
 '/services',
 '#guest-header-become-a-host-menu',
 '#simple-header-locale-menu-combined',
 '#guest-header-dropdownmenu',
 '/help',
 '/co-hosts/home',
 '/giftcards',
 '/signup_login',
 '#']

### Using GPT-4o-mini figure out which links are relevant
using "one shot prompting" to determine relevant link

In [28]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [29]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [30]:
print(get_links_user_prompt(airbnb))

Here is the list of links on the website of https://www.airbnb.com/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
#site-content
/
/homes
/experiences
/services
#guest-header-become-a-host-menu
#simple-header-locale-menu-combined
#guest-header-dropdownmenu
/help
/co-hosts/home
/giftcards
/signup_login
#


In [31]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [32]:
airbnb = Website("https://www.airbnb.com/")
airbnb.links

['#site-content',
 '/',
 '/homes',
 '/experiences',
 '/services',
 '#guest-header-become-a-host-menu',
 '#simple-header-locale-menu-combined',
 '#guest-header-dropdownmenu',
 '/help',
 '/co-hosts/home',
 '/giftcards',
 '/signup_login',
 '#']

In [33]:
get_links("https://www.airbnb.com/")

{'links': [{'type': 'about page', 'url': 'https://www.airbnb.com/about'},
  {'type': 'services page', 'url': 'https://www.airbnb.com/services'},
  {'type': 'help page', 'url': 'https://www.airbnb.com/help'}]}

## make the brochure!

Assemble all the details into another prompt to GPT4-o

In [36]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [37]:
print(get_all_details("https://www.airbnb.com/"))

Found links: {'links': [{'type': 'about page', 'url': 'https://www.airbnb.com/about'}, {'type': 'services page', 'url': 'https://www.airbnb.com/services'}, {'type': 'help page', 'url': 'https://www.airbnb.com/help'}]}
Landing page:
Webpage Title:
Airbnb: Vacation Rentals, Cabins, Beach Houses, Unique Homes & Experiences
Webpage Contents:
Skip to content
We’re sorry, some parts of the Airbnb website don’t work properly without JavaScript enabled.
Airbnb homepage
Homes
Homes
Experiences
Experiences, new
Services
Services, new
Where
Date
Add dates
Who
Add guests
Become a host
Help Center
Become a host
It's easy to start hosting and earn extra income.
Find a co-host
Gift cards
Log in or sign up
Homes in New York
Homes in Paris



about page
Webpage Title:
About us - Airbnb Newsroom
Webpage Contents:
Skip to main content
Airbnb
Newsroom
Toggle navigation
About us
Media assets
Product releases
Contact
English
English
AU and NZ
India
Malaysia
Ireland
United Kingdom
Čeština
Dansk
Deutsch
Catal

In [45]:


# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information. "

In [46]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [47]:
get_brochure_user_prompt("airbnb", "https://www.airbnb.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.airbnb.com/about'}, {'type': 'services page', 'url': 'https://www.airbnb.com/services'}, {'type': 'help page', 'url': 'https://www.airbnb.com/help'}, {'type': 'sign up/login page', 'url': 'https://www.airbnb.com/signup_login'}]}


"You are looking at a company called: airbnb\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nAirbnb: Vacation Rentals, Cabins, Beach Houses, Unique Homes & Experiences\nWebpage Contents:\nSkip to content\nWe’re sorry, some parts of the Airbnb website don’t work properly without JavaScript enabled.\nAirbnb homepage\nHomes\nHomes\nExperiences\nExperiences, new\nServices\nServices, new\nWhere\nDate\nAdd dates\nWho\nAdd guests\nBecome a host\nHelp Center\nBecome a host\nIt's easy to start hosting and earn extra income.\nFind a co-host\nGift cards\nLog in or sign up\nHomes in New York\nHomes in Paris\n\n\n\nabout page\nWebpage Title:\nAbout us - Airbnb Newsroom\nWebpage Contents:\nSkip to main content\nAirbnb\nNewsroom\nToggle navigation\nAbout us\nMedia assets\nProduct releases\nContact\nEnglish\nEnglish\nAU and NZ\nIndia\nMalaysia\nIreland\nUnited Kingdom\nČeštin

In [50]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [51]:
stream_brochure("airbnb", "https://www.airbnb.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.airbnb.com/about'}, {'type': 'services page', 'url': 'https://www.airbnb.com/services'}, {'type': 'careers page', 'url': 'https://www.airbnb.com/careers'}]}


# Welcome to Airbnb: Your Ticket to Unique Stays and Experiences!

**Who’s Your Host?**  
Airbnb started back in 2007 when two hosts welcomed three fun-loving guests into their cozy San Francisco home. Since then, we’ve grown faster than a caffeine-fueled squirrel! We now boast over **5 million hosts** who’ve welcomed more than **2 billion guests** across **220 countries**. Yes, you heard that right -**BILLIONS**!

---

### **Why Choose Airbnb?**
- **Unique Rentals:** From treehouses to entire castles (that’s right, your very own kingdom!), our listings are as diverse as the guests who book them. 
- **Experiences Like No Other:** Go beyond the ordinary! Tap into ‘Experiences’ for local treasures such as llama yoga in the park or a cooking class with someone’s grandma (cooking skills not guaranteed).
- **Join the Millionaires Club:** Hosts have collectively earned over **$300 billion**! Why not put your spare room to work and earn some side cash? 

---

### **The Airbnb Culture: Community, Growth, and Giggles!**

At Airbnb, we believe in thick blankets, creative minds and flexible working hours (pajamas encourage creativity, right?). Our team fosters a friendly, laugh-filled environment where new ideas are welcomed with open arms – and possibly a dance-off!

- **Diversity? We’ve Got It:** With listings available in **150k+ cities and towns**, it’s hard not to find a home that screams “pick me!” Whether it's a beach house in Bali or a snug cabin in the Rockies, there’s something for everyone!
- **Authentic Connections:** We’re like that friend who always knows a guy. Our platform helps you connect with the heart of a community, experience local cultures and probably gain a few new pen pals along the way.

---

### **Careers at Airbnb: Join the Fun!**

Looking for a place where you can wear your “I love to travel” shirt while doing what you love? We’re on the lookout for quirky, passionate individuals to join our team!

- **Get on Board:** From tech wizards to marketing maestros, if you think you can add some sunshine to our community, we’d love to hear from you.
- **Flexible Life:** Work where you love to host! Go ahead, log in from your treehouse Airbnb - just ensure the Wi-Fi is good!

---

### **Join the Airbnb Family!**
So, what are you waiting for? Jump into the world of Airbnb where every stay is an adventure waiting to happen, and who knows, you might just end up being the next world-renowned host! 

Book your next adventure, host your own home, or become part of a community that feels more like family. Because life's too short to stay in boring hotels!

**Remember:** At Airbnb, we don’t just find you a place to stay; we help you create memories that will warm your heart (and maybe even bring a tear to your eye).  

---

**Airbnb: Where your next adventure awaits!!** 🌍❤️🏠